# Load and install libraries

In [1]:
#Load libraries
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.api import ExponentialSmoothing, SimpleExpSmoothing, Holt

%matplotlib inline

# Import Kaggle data

In [ ]:
%%bash
pip install dask_xgboost
pip install kaggle
export KAGGLE_USERNAME=ellastar261
export KAGGLE_KEY=92c0b8137c386da6775be167e8e8ca09

kaggle competitions download -c m5-forecasting-accuracy

unzip -n m5-forecasting-accuracy -d m5-forecasting-accuracy
rm -rf sample_data

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/

Archive:  m5-forecasting-accuracy.zip
  inflating: m5-forecasting-accuracy/calendar.csv  
  inflating: m5-forecasting-accuracy/sales_train_evaluation.csv  
  inflating: m5-forecasting-accuracy/sales_train_validation.csv  
  inflating: m5-forecasting-accuracy/sample_submission.csv  
  inflating: m5-forecasting-accuracy/sell_prices.csv  


100%|██████████| 45.8M/45.8M [00:02<00:00, 21.0MB/s]


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#input paths for base data
trainpath = '/content/drive/MyDrive/Capstone/Models/m5-forecasting-accuracy/sales_train_evaluation.csv'
calpath = '/content/drive/MyDrive/Capstone/Models/m5-forecasting-accuracy/calendar.csv'

In [4]:
end_train_week = 264
prediction_horizon = 12

In [5]:
#Read data into 
train_df = pd.read_csv(trainpath)
cal = pd.read_csv(calpath)

In [6]:
index_columns = ['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id']
grid_df = pd.melt(train_df, id_vars=index_columns, var_name='d', value_name='demand')

In [7]:
weekly_df = grid_df[['id','demand','d']].merge(cal[['d','wm_yr_wk']],on=['d'],how='left')
weekly_df = weekly_df[weekly_df.wm_yr_wk<=11616]
weekly_df = weekly_df.groupby(['id','wm_yr_wk']).sum().reset_index()
week_df = pd.DataFrame(data={'wm_yr_wk':weekly_df.wm_yr_wk.unique(),'week':range(len(weekly_df.wm_yr_wk.unique()))})
weekly_df = weekly_df.merge(week_df, on=['wm_yr_wk'], how='left')
weekly_df = weekly_df[['id','demand','week']].pivot(index='id', columns='week', values='demand').reset_index()

In [ ]:
del week_df, grid_df, train_df, cal

In [8]:
weekly_df

week,id,0,1,2,3,4,5,6,7,8,...,267,268,269,270,271,272,273,274,275,276
0,FOODS_1_001_CA_1_evaluation,10,6,10,13,15,11,7,6,5,...,8,9,9,9,6,9,3,12,4,5
1,FOODS_1_001_CA_2_evaluation,11,12,12,24,3,6,10,10,7,...,7,3,3,4,3,20,7,5,5,3
2,FOODS_1_001_CA_3_evaluation,8,9,17,6,18,19,6,8,8,...,3,11,13,2,7,13,11,3,2,8
3,FOODS_1_001_CA_4_evaluation,5,3,3,5,4,8,5,5,5,...,2,3,3,2,0,3,3,1,2,2
4,FOODS_1_001_TX_1_evaluation,2,4,0,5,4,0,4,12,6,...,5,7,0,1,0,0,2,4,6,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_evaluation,2,4,5,1,0,0,3,1,0,...,4,3,3,1,2,3,0,1,1,1
30486,HOUSEHOLD_2_516_TX_3_evaluation,2,0,2,1,2,1,2,1,4,...,0,0,2,2,1,0,0,3,1,2
30487,HOUSEHOLD_2_516_WI_1_evaluation,1,0,2,1,1,1,1,0,1,...,0,1,0,0,1,0,2,0,2,1
30488,HOUSEHOLD_2_516_WI_2_evaluation,0,0,0,0,0,2,0,0,0,...,0,0,0,0,0,1,0,0,0,2


In [9]:
weekly_df = weekly_df.loc[:,:end_train_week]

In [10]:
weekly_df

week,id,0,1,2,3,4,5,6,7,8,...,255,256,257,258,259,260,261,262,263,264
0,FOODS_1_001_CA_1_evaluation,10,6,10,13,15,11,7,6,5,...,5,3,4,2,2,5,8,6,5,3
1,FOODS_1_001_CA_2_evaluation,11,12,12,24,3,6,10,10,7,...,5,9,8,5,8,6,5,9,10,13
2,FOODS_1_001_CA_3_evaluation,8,9,17,6,18,19,6,8,8,...,4,12,0,20,0,4,2,5,19,5
3,FOODS_1_001_CA_4_evaluation,5,3,3,5,4,8,5,5,5,...,4,2,1,1,1,1,0,0,0,0
4,FOODS_1_001_TX_1_evaluation,2,4,0,5,4,0,4,12,6,...,1,0,28,10,7,0,1,5,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,HOUSEHOLD_2_516_TX_2_evaluation,2,4,5,1,0,0,3,1,0,...,2,2,3,1,1,2,0,1,2,4
30486,HOUSEHOLD_2_516_TX_3_evaluation,2,0,2,1,2,1,2,1,4,...,3,1,1,1,2,4,0,0,0,3
30487,HOUSEHOLD_2_516_WI_1_evaluation,1,0,2,1,1,1,1,0,1,...,1,0,0,2,0,1,1,0,0,1
30488,HOUSEHOLD_2_516_WI_2_evaluation,0,0,0,0,0,2,0,0,0,...,0,1,1,0,0,1,1,2,0,1


In [ ]:
ids = list(weekly_df['id'])

In [ ]:
results = pd.DataFrame()
for i in range(len(ids)):
  print(i)
  id = ids[i]
  series = np.array(weekly_df.iloc[i,1:]) 
  fit1 = ExponentialSmoothing(series, seasonal_periods=52, trend='add', seasonal='add').fit(use_boxcox=False) 
  adddf = pd.DataFrame(data={'id':ids[i],'demand':list(fit1.forecast(prediction_horizon)),'week':[i for i in range(end_train_week+1,end_train_week+prediction_horizon+1)]})
  results = pd.concat([results,adddf],axis=0)
results

0
1


/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/holtwinters/model.py:429: FutureWarning: After 0.13 initialization must be handled at model creation
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/holtwinters/model.py:1116: FutureWarning: Setting use_boxcox during fit has been deprecated and will be removed after 0.13. It must be set during model initialization.
  FutureWarning,


Streaming output truncated to the last 5000 lines.
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069
2070
2071
2072
2073
2074
2075
2076
2077
2078
2079
2080
2081
2082
2083
2084
2085
2086
2087
2088
2089
2090
2091
2092
2093
2094
2095
2096
2097
2098
2099
2100
2101
2102
2103
2104
2105
2106
2107
2108
2109
2110
2111
2112
2113
2114
2115
2116
2117
2118
2119
2120
2121
2122
2123
2124
2125
2126
2127
2128
2129
2130
2131
2132
2133
2134
2135
2136
2137
2138
2139
2140
2141
2142
2143
2144
2145
2146
2147
2148
2149
2150
2151
2152
2153
2154
2155
2156
2157
2158
2159
2160
2161
2162
2163
2164
2165
2166
2167
2168
2169
2170
2171
2172
2173
2174
2175
2176
2177
2178
2179
2180
2181
2182
2183
2184
2185
2186
2187
2188
2189
2190
2191
2192
2193
2194
2195
2196
2197
2198
2199
2200
2201
2202
2203
2204
2205
2206
2207
2208
2209
2210
2211
2212

/usr/local/lib/python3.7/dist-packages/statsmodels/tsa/holtwinters/model.py:922: ConvergenceWarning: Optimization failed to converge. Check mle_retvals.
  ConvergenceWarning,


Streaming output truncated to the last 5000 lines.
25490
25491
25492
25493
25494
25495
25496
25497
25498
25499
25500
25501
25502
25503
25504
25505
25506
25507
25508
25509
25510
25511
25512
25513
25514
25515
25516
25517
25518
25519
25520
25521
25522
25523
25524
25525
25526
25527
25528
25529
25530
25531
25532
25533
25534
25535
25536
25537
25538
25539
25540
25541
25542
25543
25544
25545
25546
25547
25548
25549
25550
25551
25552
25553
25554
25555
25556
25557
25558
25559
25560
25561
25562
25563
25564
25565
25566
25567
25568
25569
25570
25571
25572
25573
25574
25575
25576
25577
25578
25579
25580
25581
25582
25583
25584
25585
25586
25587
25588
25589
25590
25591
25592
25593
25594
25595
25596
25597
25598
25599
25600
25601
25602
25603
25604
25605
25606
25607
25608
25609
25610
25611
25612
25613
25614
25615
25616
25617
25618
25619
25620
25621
25622
25623
25624
25625
25626
25627
25628
25629
25630
25631
25632
25633
25634
25635
25636
25637
25638
25639
25640
25641
25642
25643
25644
25645
25646
25647
2

,id,demand,week
0,FOODS_1_001_CA_1_evaluation,6.529515,264
1,FOODS_1_001_CA_1_evaluation,3.529567,265
2,FOODS_1_001_CA_1_evaluation,3.329686,266
3,FOODS_1_001_CA_1_evaluation,4.329619,267
4,FOODS_1_001_CA_1_evaluation,3.129589,268
...,...,...,...
7,HOUSEHOLD_2_516_WI_3_evaluation,0.998196,271
8,HOUSEHOLD_2_516_WI_3_evaluation,0.798221,272
9,HOUSEHOLD_2_516_WI_3_evaluation,0.398236,273
10,HOUSEHOLD_2_516_WI_3_evaluation,0.398223,274


In [ ]:
fcast = results.pivot(index='id',columns='week',values='demand').reset_index()

In [ ]:
#write to CSV
fcast.to_csv('weekly_exp_smooth.csv', index=False) 